In [1]:
%cd /content/drive/MyDrive/HKU_OSDG/

/content/drive/MyDrive/HKU_OSDG


In [2]:
import pandas as pd
#this excel file has already been cleaned via the rule: positive>negative and agreement>0.6
df_osdg=pd.read_excel("Complete-OSDG-dataset.xlsx").drop(columns = ['Unnamed: 0'], axis = 1)
df_osdg

,doi,text_id,text,sdg,labels_negative,labels_positive,agreement
0,10.6027/9789289342698-7-en,00021941702cd84171ff33962197ca1f,"From a gender perspective, Paulgaard points ou...",5,1,7,0.750000
1,10.1787/9789264289062-4-en,0004eb64f96e1620cd852603d9cbe4d4,The average figure also masks large difference...,3,1,6,0.714286
2,10.1787/9789264117563-8-en,000bfb17e9f3a00d4515ab59c5c487e7,The Israel Oceanographic and Limnological Rese...,6,0,3,1.000000
3,10.18356/805b1ae4-en,001180f5dd9a821e651ed51e30d0cf8c,Previous chapters have discussed ways to make ...,2,0,3,1.000000
4,10.1787/9789264310278-en,001f1aee4013cb098da17a979c38bc57,Prescription rates appear to be higher where l...,8,0,3,1.000000
...,...,...,...,...,...,...,...
17243,10.18356/dd2e120a-en,ffeb4e2b3d0f4d449053b1b15f51e4eb,"In Mauritius, housing development for the econ...",11,0,4,1.000000
17244,10.18356/83efcfeb-en,ffef3ec4ea192cdfed2ca43d4b2208a9,"In consequence, the number of urban dwellers p...",3,0,4,1.000000
17245,10.1787/9789264214033-7-en,fff9b6b75bde5397ba26ed1b163ad306,"It also features individual accountability, wh...",4,0,8,1.000000
17246,10.1787/9789264186378-8-en,fffe58dae29de6b843aff09b6e2f77f0,Groundwater quality can also be affected by co...,6,0,3,1.000000


In [3]:
#convert the label format to multi-label format, similar to the label style of HKU dataset
df_osdg[["sdg"]]=df_osdg[["sdg"]].values.astype(str)
one_hot=["sdg"] 
sdg_coded=pd.get_dummies(df_osdg[one_hot])
for i in sdg_coded.columns.values.tolist():
    df_osdg[i]=sdg_coded[i]
df_osdg=df_osdg.drop(columns = one_hot, axis = 1)
df_osdg=df_osdg.drop(columns=["doi","text_id","labels_negative","labels_positive","agreement"])
df_osdg

,text,sdg_1,sdg_10,sdg_11,sdg_12,sdg_13,sdg_14,sdg_15,sdg_2,sdg_3,sdg_4,sdg_5,sdg_6,sdg_7,sdg_8,sdg_9
0,"From a gender perspective, Paulgaard points ou...",0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,The average figure also masks large difference...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2,The Israel Oceanographic and Limnological Rese...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
3,Previous chapters have discussed ways to make ...,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,Prescription rates appear to be higher where l...,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17243,"In Mauritius, housing development for the econ...",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
17244,"In consequence, the number of urban dwellers p...",0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
17245,"It also features individual accountability, wh...",0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
17246,Groundwater quality can also be affected by co...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [4]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
#Remove all symbols, uppercase to lowercase
cleaned_text=[]
not_alpha=[]
for i in df_osdg["text"]:
    i=i.lower()
    for j in i:
        if not str.isalpha(j) and j !=' ':
                i=i.replace(j,"")
    cleaned_text.append(i)
df_osdg["text"]=cleaned_text
X=df_osdg["text"]
Y=df_osdg[["sdg_1","sdg_2","sdg_3","sdg_4","sdg_5","sdg_6","sdg_7","sdg_8","sdg_9","sdg_10","sdg_11","sdg_12","sdg_13","sdg_14","sdg_15"]]
Y=Y.rename(columns={"sdg_1": 'SDG1', "sdg_2": 'SDG2',"sdg_3": 'SDG3', "sdg_4": 'SDG4',"sdg_5": 'SDG5', "sdg_6": 'SDG6',"sdg_7": 'SDG7', 
          "sdg_8": 'SDG8',"sdg_9": 'SDG9', "sdg_10": 'SDG10',"sdg_11": 'SDG11', "sdg_12": 'SDG12',"sdg_13": 'SDG13', "sdg_14": 'SDG14',"sdg_15": 'SDG15'})
Y

,SDG1,SDG2,SDG3,SDG4,SDG5,SDG6,SDG7,SDG8,SDG9,SDG10,SDG11,SDG12,SDG13,SDG14,SDG15
0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17243,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
17244,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
17245,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
17246,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [5]:
#for data analysis and modeling
import tensorflow as tf
from tensorflow.keras.layers import LSTM, GRU, Dense, Embedding, Dropout
from tensorflow.keras.preprocessing import text, sequence 
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
#for text cleaning
import string
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
#for visualization
import matplotlib.pyplot as plt
#xtrain, xtest, ytrain, ytest = train_test_split(X, Y, shuffle=True, test_size=0.2)
# find the length of the largest sentence in training data
#max_len = xtrain.apply(lambda x: len(x)).max()
#print(f'Max number of words in a text in training data: {max_len}')

In [6]:
from copy import deepcopy
def convert_res_format(y_pred,ytest):#convert y_pred to the same format as ytest
  prediction=deepcopy(ytest)
  for col in prediction.columns:
      prediction[col].values[:] = 0
  indexes=prediction.index.tolist()
  for i in range(len(y_pred)):
    for j in range(15):
      if y_pred[i][j]>0.5:
        prediction.loc[indexes[i],"SDG"+str(j+1)]=1
  return prediction

from multieva import evaluate_on_each_SDG
def evaluate(ytest,prediction):
  ytest=ytest.reset_index(drop=True)
  prediction=prediction.reset_index(drop=True)
  return evaluate_on_each_SDG(ytest,prediction)

In [7]:
embedding_vectors = {}
with open("glove.6B.100d.txt", encoding='utf-8') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embedding_vectors[word] = coefs
print(f"Size of vocabulary in GloVe: {len(embedding_vectors)}")   

Size of vocabulary in GloVe: 400000


In [60]:
#use pretrained word embedding vectors
def train(xtrain,xtest,ytrain,ytest):
  max_words = 1000
  tokenizer = text.Tokenizer(num_words = max_words)
  # create the vocabulary by fitting on x_train text
  tokenizer.fit_on_texts(xtrain)
  # generate the sequence of tokens
  xtrain_seq = tokenizer.texts_to_sequences(xtrain)
  xtest_seq = tokenizer.texts_to_sequences(xtest)
  max_len = xtrain.apply(lambda x: len(x)).max()
  print("max_len=",max_len)
  # pad the sequences
  xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
  xtest_pad = sequence.pad_sequences(xtest_seq, maxlen=max_len)
  word_index = tokenizer.word_index

  # print('Text Example:', xtrain[0])
  # print('Sequence of indices(before padding):', xtrain_seq[0])
  # print('Sequence of indices(after padding):', xtrain_pad[0])
  #initialize the embedding_matrix with zeros
  emb_dim = 100
  if max_words is not None: 
      vocab_len = max_words 
  else:
      vocab_len = len(word_index)+1
  embedding_matrix = np.zeros((vocab_len, emb_dim))
  oov_count = 0
  oov_words = []
  for word, idx in word_index.items():
      if idx < vocab_len:
          embedding_vector = embedding_vectors.get(word)
          if embedding_vector is not None:
              embedding_matrix[idx] = embedding_vector
          else:
              oov_count += 1 
              oov_words.append(word)
  #print some of the out of vocabulary words
  #print(f'Some out of valubulary words: {oov_words[0:5]}')
  #create model
  emb_dim = embedding_matrix.shape[1]
  gru_model = Sequential()
  gru_model.add(Embedding(vocab_len, emb_dim, trainable = False, weights=[embedding_matrix]))
  gru_model.add(GRU(32, return_sequences=False))
  gru_model.add(Dropout(0.3))
  gru_model.add(Dense(15, activation = 'sigmoid')) #notice sigmoid is used here, but not softmax
  gru_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  #print(gru_model.summary())

  tf.config.list_physical_devices('GPU')
  batch_size = 400; epochs = 100
  history = gru_model.fit(xtrain_pad, np.asarray(ytrain), validation_data=(xtest_pad, np.asarray(ytest)), batch_size = 400, epochs = 100)

  # plt.figure(figsize=(15, 7))
  # plt.plot(range(epochs), history.history['accuracy'])
  # plt.plot(range(epochs), history.history['val_accuracy'])
  # plt.legend(['training_acc', 'validation_acc'])
  # plt.title('Accuracy')
  y_pred=gru_model.predict(xtest_pad)
  
  prediction=convert_res_format(y_pred,ytest)
  eva_res=evaluate(ytest,prediction)
  return eva_res,tokenizer,gru_model

In [9]:
from sklearn.model_selection import KFold
from tqdm import tqdm
kf = KFold(n_splits=10)
kf.get_n_splits(X)
evaluation_results=[]
for train_index, test_index in tqdm(kf.split(X)):
    xtrain, xtest = X.loc[train_index], X.loc[test_index]
    ytrain, ytest = Y.loc[train_index], Y.loc[test_index]
    eva_res,tokenizer,gru_model=train(xtrain,xtest,ytrain,ytest)
    evaluation_results.append(eva_res)


0it [00:00, ?it/s]

Epoch 1/100
39/39 [==============================] - 6s 72ms/step - loss: 0.4826 - accuracy: 0.0669 - val_loss: 0.3007 - val_accuracy: 0.0910
Epoch 2/100
39/39 [==============================] - 2s 59ms/step - loss: 0.2850 - accuracy: 0.0806 - val_loss: 0.2389 - val_accuracy: 0.1206
Epoch 3/100
39/39 [==============================] - 2s 60ms/step - loss: 0.2541 - accuracy: 0.1198 - val_loss: 0.2292 - val_accuracy: 0.2852
Epoch 4/100
39/39 [==============================] - 2s 61ms/step - loss: 0.2440 - accuracy: 0.1693 - val_loss: 0.2216 - val_accuracy: 0.2672
Epoch 5/100
39/39 [==============================] - 2s 60ms/step - loss: 0.2346 - accuracy: 0.2404 - val_loss: 0.2124 - val_accuracy: 0.3119
Epoch 6/100
39/39 [==============================] - 2s 61ms/step - loss: 0.2249 - accuracy: 0.3006 - val_loss: 0.2023 - val_accuracy: 0.4371
Epoch 7/100
39/39 [==============================] - 2s 60ms/step - loss: 0.2143 - accuracy: 0.3532 - val_loss: 0.1914 - val_accuracy: 0.4777
Epoch 

1it [04:20, 260.68s/it]

Epoch 1/100
39/39 [==============================] - 4s 72ms/step - loss: 0.4608 - accuracy: 0.0999 - val_loss: 0.2927 - val_accuracy: 0.0788
Epoch 2/100
39/39 [==============================] - 2s 61ms/step - loss: 0.2750 - accuracy: 0.1145 - val_loss: 0.2367 - val_accuracy: 0.1675
Epoch 3/100
39/39 [==============================] - 2s 62ms/step - loss: 0.2477 - accuracy: 0.1544 - val_loss: 0.2273 - val_accuracy: 0.2922
Epoch 4/100
39/39 [==============================] - 2s 61ms/step - loss: 0.2375 - accuracy: 0.2098 - val_loss: 0.2179 - val_accuracy: 0.3710
Epoch 5/100
39/39 [==============================] - 2s 62ms/step - loss: 0.2255 - accuracy: 0.2957 - val_loss: 0.2045 - val_accuracy: 0.4186
Epoch 6/100
39/39 [==============================] - 2s 62ms/step - loss: 0.2107 - accuracy: 0.3816 - val_loss: 0.1906 - val_accuracy: 0.5067
Epoch 7/100
39/39 [==============================] - 2s 63ms/step - loss: 0.1972 - accuracy: 0.4442 - val_loss: 0.1774 - val_accuracy: 0.5472
Epoch 

2it [08:38, 259.26s/it]

...

8it [35:59, 268.55s/it]

Epoch 1/100
39/39 [==============================] - 4s 73ms/step - loss: 0.4940 - accuracy: 0.0681 - val_loss: 0.2967 - val_accuracy: 0.1201
Epoch 2/100
39/39 [==============================] - 2s 61ms/step - loss: 0.2834 - accuracy: 0.0924 - val_loss: 0.2394 - val_accuracy: 0.1398
Epoch 3/100
39/39 [==============================] - 2s 61ms/step - loss: 0.2546 - accuracy: 0.1169 - val_loss: 0.2307 - val_accuracy: 0.2204
Epoch 4/100
39/39 [==============================] - 2s 62ms/step - loss: 0.2449 - accuracy: 0.1573 - val_loss: 0.2228 - val_accuracy: 0.2749
Epoch 5/100
39/39 [==============================] - 2s 61ms/step - loss: 0.2342 - accuracy: 0.2287 - val_loss: 0.2118 - val_accuracy: 0.3788
Epoch 6/100
39/39 [==============================] - 2s 62ms/step - loss: 0.2213 - accuracy: 0.3102 - val_loss: 0.1999 - val_accuracy: 0.4611
Epoch 7/100
39/39 [==============================] - 2s 62ms/step - loss: 0.2104 - accuracy: 0.3706 - val_loss: 0.1894 - val_accuracy: 0.5122
Epoch 

9it [40:20, 266.03s/it]

Epoch 1/100
39/39 [==============================] - 5s 74ms/step - loss: 0.4665 - accuracy: 0.0512 - val_loss: 0.2926 - val_accuracy: 0.0423
Epoch 2/100
39/39 [==============================] - 2s 63ms/step - loss: 0.2800 - accuracy: 0.0660 - val_loss: 0.2418 - val_accuracy: 0.1108
Epoch 3/100
39/39 [==============================] - 2s 62ms/step - loss: 0.2526 - accuracy: 0.1214 - val_loss: 0.2316 - val_accuracy: 0.2169
Epoch 4/100
39/39 [==============================] - 2s 63ms/step - loss: 0.2415 - accuracy: 0.1809 - val_loss: 0.2235 - val_accuracy: 0.2958
Epoch 5/100
39/39 [==============================] - 2s 63ms/step - loss: 0.2311 - accuracy: 0.2521 - val_loss: 0.2126 - val_accuracy: 0.4298
Epoch 6/100
39/39 [==============================] - 2s 63ms/step - loss: 0.2200 - accuracy: 0.3268 - val_loss: 0.2010 - val_accuracy: 0.4954
Epoch 7/100
39/39 [==============================] - 3s 76ms/step - loss: 0.2081 - accuracy: 0.3973 - val_loss: 0.1891 - val_accuracy: 0.5261
Epoch 

10it [44:43, 268.33s/it]


In [22]:
sum_df=evaluation_results[0]
for i in range(1,len(evaluation_results)):
  try:
    sum_df = sum_df+ evaluation_results[i]
  except:
    print("error") #p=0,r=0 or tp=fp=0, something like that.
avg_df=sum_df/5
avg_df #the average schore of the cross validation(averaged 5/10 folds only)

error
error
error
error
error


,Accuracy,Precision,Recall,F1
SDG1,0.976232,0.864733,0.759747,0.808475
SDG2,0.977971,0.835974,0.646932,0.728852
SDG3,0.976,0.915356,0.864338,0.888916
SDG4,0.981333,0.942738,0.91593,0.929108
SDG5,0.978899,0.926733,0.914489,0.920349
SDG6,0.983536,0.90762,0.870244,0.888152
SDG7,0.976696,0.898949,0.838785,0.867471
SDG8,0.961623,0.699983,0.418099,0.521033
SDG9,0.967072,0.748233,0.318369,0.442631
SDG10,0.979014,0.895873,0.183718,0.297757


In [61]:
#another experiment: 80％training, 20％evaluating
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, shuffle=True, test_size=0.2)
eva_res,tokenizer,gru_model=train(xtrain,xtest,ytrain,ytest)
eva_res

max_len= 1276
Epoch 1/50
35/35 [==============================] - 4s 75ms/step - loss: 0.4212 - accuracy: 0.0599 - val_loss: 0.2767 - val_accuracy: 0.0626
Epoch 2/50
35/35 [==============================] - 2s 62ms/step - loss: 0.2742 - accuracy: 0.0873 - val_loss: 0.2395 - val_accuracy: 0.1922
Epoch 3/50
35/35 [==============================] - 2s 62ms/step - loss: 0.2525 - accuracy: 0.1323 - val_loss: 0.2306 - val_accuracy: 0.2461
Epoch 4/50
35/35 [==============================] - 2s 63ms/step - loss: 0.2427 - accuracy: 0.1751 - val_loss: 0.2230 - val_accuracy: 0.3614
Epoch 5/50
35/35 [==============================] - 2s 63ms/step - loss: 0.2323 - accuracy: 0.2395 - val_loss: 0.2133 - val_accuracy: 0.4209
Epoch 6/50
35/35 [==============================] - 2s 66ms/step - loss: 0.2207 - accuracy: 0.3156 - val_loss: 0.2018 - val_accuracy: 0.4762
Epoch 7/50
35/35 [==============================] - 3s 77ms/step - loss: 0.2086 - accuracy: 0.3985 - val_loss: 0.1891 - val_accuracy: 0.4994

,Accuracy,Precision,Recall,F1
SDG1,0.975072,0.903382,0.73913,0.813043
SDG2,0.975942,0.917431,0.574713,0.706714
SDG3,0.977101,0.912281,0.864266,0.887624
SDG4,0.977101,0.939462,0.889597,0.91385
SDG5,0.983478,0.951389,0.919463,0.935154
SDG6,0.983768,0.900398,0.879377,0.889764
SDG7,0.974783,0.895425,0.832827,0.862992
SDG8,0.963478,0.775862,0.28481,0.416667
SDG9,0.96058,1.0,0.007299,0.014493
SDG10,0.975072,tp=fp=0,0.0,"p=0,r=0"


In [64]:
#here we make a fake sample xtest[0], and hope that model can give a multi-label result on it.
xtest[0]=xtest[6277]+xtest[11039]
xtest

6277     populations of schoolage children have shrunk ...
11039    restricting investments in generating capacity...
8092     overall meanstested family and education benef...
7815     the  recommendation complements and deepens th...
1262     in burundi for example  per cent or less of wo...
                               ...                        
11827    other factors associated with inwork poverty i...
10887    this led to a demand for licensed electricians...
12427    school feeding programmes are critical to prov...
15981    these results are noteworthy in that they are ...
0        populations of schoolage children have shrunk ...
Name: text, Length: 3451, dtype: object

In [65]:
xtest_seq = tokenizer.texts_to_sequences(xtest)
xtest_seq
xtest_pad = sequence.pad_sequences(xtest_seq, maxlen=1276)
xtest_pad

array([[  0,   0,   0, ...,   6,  77, 515],
       [  0,   0,   0, ..., 405,  14, 378],
       [  0,   0,   0, ...,   5,  82, 786],
       ...,
       [  0,   0,   0, ...,   2, 125,  13],
       [  0,   0,   0, ...,  29, 176,  27],
       [  0,   0,   0, ..., 405,  14, 378]], dtype=int32)

In [68]:
print(ytest.loc[6277]) #label is SDG4
print("\n")
print(ytest.loc[11039]) #label is SDG7

SDG1     0
SDG2     0
SDG3     0
SDG4     1
SDG5     0
SDG6     0
SDG7     0
SDG8     0
SDG9     0
SDG10    0
SDG11    0
SDG12    0
SDG13    0
SDG14    0
SDG15    0
Name: 6277, dtype: uint8


SDG1     0
SDG2     0
SDG3     0
SDG4     0
SDG5     0
SDG6     0
SDG7     1
SDG8     0
SDG9     0
SDG10    0
SDG11    0
SDG12    0
SDG13    0
SDG14    0
SDG15    0
Name: 11039, dtype: uint8


In [67]:
y_test=gru_model.predict(xtest_pad)
y_sample1=y_test[0] #xtest[6277], label is SDG4
y_sample2=y_test[1] #xtest[11039], label is SDG7
y_sample3=y_test[-1] #fake sample, xtest[0]=xtest[6277]+xtest[11039], label is SDG4 and SDG7
print(y_sample1)#prediction is SDG4
print("===================")
print(y_sample2)#prediction is SDG7
print("===================")
print(y_sample3)#SDG7 > SDG4 >>>others 数量级的差异

108/108 [==============================] - 2s 16ms/step
[2.7402197e-03 1.4423982e-03 1.1951574e-03 9.7563541e-01 3.5374656e-03
 9.7636384e-04 1.9957088e-03 5.8506043e-03 6.1018490e-03 9.5069420e-04
 3.1772163e-03 1.4957194e-03 2.4030883e-04 2.6742124e-03 3.8206524e-03]
[3.6758396e-03 3.6478860e-03 2.8646253e-03 1.6025711e-03 1.1081869e-03
 4.6721199e-03 9.5856690e-01 4.4520027e-03 1.7334331e-02 1.6114418e-03
 1.9364016e-03 9.9597415e-03 1.7986855e-02 2.3545476e-03 7.1265380e-04]
[6.5766275e-03 3.9895959e-03 8.2287192e-03 1.6071740e-01 4.5171496e-04
 2.9186958e-03 7.6386780e-01 4.0726974e-03 1.3098907e-02 1.2229535e-03
 3.6203091e-03 6.5985867e-03 2.0927656e-03 2.0522890e-03 8.0864708e-04]


reflection:  
It is expected that both confidence rates of SDG4 and SDG7 can be greater than 50%, but the confidence rate for SDG4 was less than 50%, although still much greater than the confidence rates for the other categories. Perhaps to some extent the information of SDG4, which is spliced in the front half of the fake sample, is lost due to the long distance dependence.
